# Build Synapses

**Contents**

- [brainpy.SynType](#brainpy.SynType)
- [brainpy.SynConn](#brainpy.SynConn)
- [brainpy.connectivity](#brainpy.connectivity)

Same with the neuron models, the *definition* and *usage* of the synapse model are separated from each other. Specifically, two classes should be used:

- ``brainpy.SynType``: Define the abstract synapse model.
- ``brainpy.SynConn``: Use the abstract synapse model to generate a concrete synapse connection.

The synapse model should consider two things:

- The ``connection`` between neuron groups: how the pre-synaptic neurons connect to the post-synaptic neurons.
- What ``computation`` is used on each synaptic connection. 

In the next, I will illustrate how to define a synaptic model by using ``brainpy.SynType``, and then tell you how to use your defined model with ``brainpy.SynConn``. 

Before we start, let's import your favorite BrainPy and Numpy packages.

In [1]:
import brainpy as bp
import numpy as np

## brainpy.SynType

### Support for `SynType` definition

Same with the neuron models, BrainPy provides several interfaces to support the SynType definition. Users can combine them arbitrarily to define a synapse model. Specifically, the supports include:

- **Numerical integration of [differential equations](https://brainpy.readthedocs.io/en/latest/advanced/differential_equations.html)**: by using the decorator `@brainpy.integrate`, users can define an integrator for a ODE or SDE. The numerical method for integration can be chosen from the supported [methods](https://brainpy.readthedocs.io/en/latest/advanced/numerical_integrators.html).

- **Synaptic state management**: BrainPy provides `brainpy.SynState` for convenient synapse state management. 

- **Automatical synaptic delay**: The synapse modeling usually includes a delay time (typically 0.3–0.5 ms) required for a neurotransmitter to be released from a presynaptic membrane, diffuse across the synaptic cleft, and bind to a receptor site on the post-synaptic membrane. BrainPy provides `@brainpy.delayed` decorator for automatical state delay. On the functions which use the delayed state, you just need to add the `@delayed` decorator on them. 

### Scalar-based Synapse Model

The updating logic of a synapse should consider the connections between groups and the computations on each connection. Fortunately, BrainPy provides a scalar-based definition paradigm, in which you only need to consider the computation on each connection. Let's first take the AMPA synapse model as an illustrating example.

The formal equations of an AMPA synapse is given by:

$$
\frac{d s}{d t}=-\frac{s}{\tau_{decay}}+\sum_{k} \delta(t-t_{j}^{k}) \quad (1) \\
I_{syn}= \bar{g}_{syn} s (V-E_{syn}) \quad (2)
$$


where $\bar{g}_{syn}$ is the maximum synaptic conductance, $s$ is the gating variable, and $V$ is the membrane potential of the postsynaptic neuron. The time constant $\tau_{decay}$ is about 2ms and the equilibrium potential $E_{syn}$ for AMPA synapse is set to 0 mV.

In [2]:
# model parameters 

tau_decay = 2.   # time constant of the dacay after synapse respond to a neurontransmitter.
g_max = .10      # Voltage-controlled conductance per unit area
                 # associated with the Sodium (Na) and Potassium (K) 
                 # ion-channels on the synapse (postsynaptic membrane).
E = 0.           # The equilibrium potentials for the synapse.

In [3]:
# use "@integrate" to define the gating variable dynamics

@bp.integrate
def ints(s, t):
    return - s / tau_decay

In [4]:
# use "SynState" to define the synapse state

ST = bp.types.SynState('s', help='AMPA synapse state.')

Based on the above definition, the update logic of the synapse model (equation (1)) from the current time point ($t$) to the next time point $(t + dt)$ can be defined as:

In [6]:
def update_by_scalar(ST, _t, pre):
    s = ints(ST['s'], _t)
    if pre['spike'] == True:
        s += 1
    ST['s'] = s

The output synaptic value onto the post-synaptic neurons (equation (2)) can be defined as:

In [7]:
@bp.delayed
def output_by_scalar(ST, post):
    I_syn = - g_max * ST['s'] * (post['V'] - E)
    post['input'] += I_syn

As you can see, the code writting is very similar to the original equations. 

Another thing we should pay attention to is that the `update()` function needs pre-synaptic neuron state `pre`, and the `output()` function requires post-synaptic neuron state `post`. If you want someone else to use this model, you can make clear statements about the data need to satisfy. Like this,

In [5]:
requires = dict(
    pre=bp.types.NeuState('spike', help='Presynaptic neuron state must have "spike" item.'),
    post=bp.types.NeuState('V', 'input', help='Postsynaptic neuron state must have "V" and "input" item.')
)

Note that, under the scalar-based synapse model, all the neuron state and the synapse state, will be treated as the scalar-based ones.

Putting together, an AMPA synapse model is defined as:

In [8]:
AMPA = bp.SynType(name='AMPA_scalar',  ST=ST, requires=requires, 
                  steps=(update_by_scalar, output_by_scalar), mode='scalar')

### Matrix-based Synapse Model

Contrary to the scalar-based synapse model, the matrix-based model means each item in the synapse state ``ST`` is a matrix with the shape of `(num_pre, num_post)`.

For example, if you have the followng connection matrix (`conn_mat`), 

<img src="../images/syn-example-conn_mat.png" width="400 px" align="left">


the updating logic of the AMPA synapse can be coded as:

In [ ]:
def update_by_mat(ST, _t, pre, conn_mat):
    s = ints(ST['s'], _t)
    # when pre-synaptic neuron generate spikes, the  
    # corresponding position in `s` will add 1.
    s += pre['spike'].reshape((-1, 1)) * conn_mat
    ST['s'] = s

Similarly, the output function of the AMPA synapse will be adapted to

In [ ]:
@bp.delayed
def output_by_mat(ST, post):
    g = g_max * np.sum(ST['s'], axis=0)
    post['input'] -= g * (post['V'] - E)

Here, in order to compute the AMPA updating logic, a new data `conn_mat` will be used. We can make a data declaration as 

In [ ]:
requires = dict(
    pre=bp.types.NeuState('spike', help='Presynaptic neuron state must have "spike" item.'),
    post=bp.types.NeuState('V', 'input', help='Postsynaptic neuron state must have "V" and "input" item.'),
    conn_mat=bp.types.MatConn(help='Connection matrix between pre- and post- synaptic neuron group.')
)

Putting together, a matrix-based AMPA synapse model is defined as

In [ ]:
AMPA = bp.SynType(name='AMPA_matrix',  ST=ST, requires=requires, 
                  steps=(update_by_mat, output_by_mat), mode='matrix')

### Vector-based Synapse Model

Three kinds of definition provided in BrainPy to define a ``SynType``: 

- ``mode = 'scalar'``: Synapse state ``ST`` represents the state of a single synapse connection. And, each item in ``ST`` is a scalar. 
- ``mode = 'vector'``:  Synapse state ``ST`` represents the state of a group of synapse connections. And each item in ``ST`` is a vector, 
- ``mode = 'matrix'``: Synapse state ``ST`` represents the state of a group of synapse connections. And each item in ``ST`` is a matrix with the shape of `(num_pre, num_post)`.

The definition logic of scalar-based models may be more straightforward than vector- and matrix- based models. We will first introduce the definition of a simple synapse model in scalar-based mode.

In matrix mode, each item in the synapse state ``ST`` is a matrix.

The differential equation part is the same as the scalar mode, and we also need a ``SynState`` and the ``NeuState`` of presynaptic and postsynaptic neurons.

In [9]:
tau_decay = 2.
g_max = .10      
E = 0.           

@bp.integrate
def ints(s, t):
    return - s / tau_decay

ST=bp.types.SynState(['s', 'g'], help='AMPA synapse state.')
pre=bp.types.NeuState(['spike'], help='Presynaptic neuron state must have "sp" item.')
post=bp.types.NeuState(['V', 'input'], help='Presynaptic neuron state must have "V" and "inp" item.')

We also need to define a connectivity matrix to specify the connectivity patterns between the presynaptic neurons and postsynaptic neurons, which can be defined with ``brainpy.types.MatConn()``. 

In [10]:
conn_mat=bp.types.MatConn()

The update and output are also similar to the scalar mode, but notice that the ``pre`` and ``post`` here are vectors, so all the operations are vectors.

In [11]:
def update(ST, _t, pre, conn_mat):
    s = ints(ST['s'], _t)
    s += pre['spike'].reshape((-1, 1)) * conn_mat
    ST['s'] = s
    ST['g'] = g_max * s

@bp.delayed
def output(ST, post):
    g = np.sum(ST['g'], axis=0)
    post['input'] -= g * (post['V'] - E)
    
AMPA = bp.SynType(name='AMPA_synapse',
                  ST=ST,
                  requires=dict(pre=pre, post=post, conn_mat=conn_mat), 
                  steps=(update, output), 
                  mode='matrix')

In vector mode, each item in the synapse state ``ST`` is a vector.

Let's look at the synaptic connections in vector form.

#### Synaptic connectivity

Suppose we have two vectors of neurons and a vector of synapses connecting the neurons within the two neuron vectors. Many different connectivities are possible, and we use $index$ to recognize different synapses.

Each synapse receives information from one presynaptic neuron, and, commonly, different synapses get inconsistent signals. Therefore, it is helpful to specify a map from the presynaptic neuron vector to the synapses vector.

For example, we have a connectivity as below:

<img src="../images/syn-example-conn_mat.png" width="400 px" align="left">

Where 1 and 0 indicate the presence and absence of synaptic connections, respectively. We can then arrange the synapses in the following manner:

<img src="../images/syn-example-pre_ids-post_ids.png" width="700 px" align="left">

We can create a ``pre2syn`` list, the indexes of this list correspond to the indexes of the presynaptic neurons vector, and the elements indicate the indexes of synapses that having connections to the neuron. Here, the first neuron connects the 3rd, 5th, and 7th neurons with synapses 0, 1, 2, so we store [0, 1, 2] as the first element of the ``pre2syn`` list. Thus, if the first neuron fire, then we can get the indexes of synapses by ``syn_ids = pre2syn[0]`` and changes the states of those synapses.

<img src="../images/syn-example-pre2syn.png" width="200 px" align="left">

Similarly, we can use a ``post2syn`` list to indicate the connections between synapses and postsynaptic neurons. The indexes of this list correspond to the indexes of the presynaptic neurons vector, and the elements indicate the indexes of synapses that having connections to the neuron.

<img src="../images/syn-example-post2syn.png" width="200 px" align="left">

We can also create a map between two neurons vectors using a ``pre2post`` list and a ``post2pre`` list.

<img src="../images/syn-example-pre2post.png" width="200 px" align="left">

<img src="../images/syn-example-post2pre.png" width="200 px" align="left">

Other mapping ways are also possible.

<img src="../images/syn-example-post_slice_syn.png" width="700 px" align="left">

<img src="../images/syn-example-pre_slice_syn.png" width="700 px" align="left">

Now let's see how to implement a vector-based synapses model by taking AMPA model as example. The formal equations of an AMPA synapse is the same as the scalar-based one:

$$I_{syn}= \bar{g}_{syn} s (V-E_{syn})$$

$$\frac{d s}{d t}=-\frac{s}{\tau_{decay}}+\sum_{k} \delta(t-t_{j}^{k})$$

where $\bar{g}_{syn}$ is the maximum synaptic conductance, $s$ is the gating variable, and $V$ is the membrane potential of the postsynaptic neuron. The time constant $\tau_{decay}$ is about 2ms and the equilibrium potential $E_{syn}$ for AMPA synapse is 0.

The differential equation part is the same as the scalar and matrix mode, and we also need a ``SynState`` and the ``NeuState`` of presynaptic and postsynaptic neurons.

In [12]:
tau_decay = 2.
g_max = .10      
E = 0.           

@bp.integrate
def ints(s, t):
    return - s / tau_decay

ST=bp.types.SynState(['s'], help='AMPA synapse state.')
pre=bp.types.NeuState(['spike'], help='Presynaptic neuron state must have "sp" item.')
post=bp.types.NeuState(['V', 'input'], help='Presynaptic neuron state must have "V" and "inp" item.')

For the mapping between synapse and neurons, BrainPy provides ``brainpy.types.ListConn``.

In [13]:
pre2syn = bp.types.ListConn()
post2syn = bp.types.ListConn()

Assume the items in the synapse state ``ST`` and neuron states ``pre`` and ``post`` are vectors, and we have the mapping lists ``pre2syn`` and ``post2syn``, the update logic of vector-based AMPA synapse model is:

In [14]:
def update(ST, _t, pre, pre2syn):
    s = ints(ST['s'], _t)

    spikeike_idx = np.where(pre['spike'] > 0.)[0]
    for i in spikeike_idx:
        syn_idx = pre2syn[i]
        s[syn_idx] += 1.

    # update values
    ST['s'] = s
    
    
@bp.delayed
def output(ST, post, post2syn):
    post_cond = np.zeros(len(post2syn), dtype=np.float_)
    for post_id, syn_ids in enumerate(post2syn):
        post_cond[post_id] = np.sum(g_max * ST['s'][syn_ids])
    post['input'] -= post_cond * (post['V'] - E)
    
AMPA_vector = bp.SynType(name='AMPA_synapse',
                         ST=ST,
                         requires=dict(pre=pre, post=post,
                                       pre2syn=pre2syn, post2syn=post2syn),
                         steps=(update, output),
                         mode='vector')

## brainpy.SynConn

Synapse connections determine the architecture of a network. A ``brainpy.SynConn`` receives the following parameters:

- ``model``: The synapse type will be used to generate a synapse connection.
- ``pre_group``: The presynaptic neuron group.
- ``post_group``: The postsynaptic neuron group.
- ``conn``: The connection method to create synaptic connectivity between the neuron groups.
- ``monitors``: The items to monitor (record the history values.)
- ``delay``: The time of the synapse delay (in milliseconds).

BrainPy pre-defines several commonly used connection methods in ``brainpy.connect``, read [Usage of connect module](https://brainpy.readthedocs.io/en/latest/advanced/usage_of_connect_module.html) for more details.

Let's take our defined AMPA model as an exmaple.

We can get pre-defined neuron models from the ``bpmodels`` package. Here we use the leaky intergrate-and-fire (LIF) model to create neuron groups

In [15]:
from bpmodels.neurons import get_LIF

LIF = get_LIF(V_rest=-65., V_reset=-65., V_th=-55.)
pre = bp.NeuGroup(LIF, 1, monitors=['spike', 'V'])
pre.ST['V'] = -65.
post = bp.NeuGroup(LIF, 1, monitors=['V'])
post.ST['V'] = -65.

In [16]:
syn = bp.SynConn(model=AMPA, pre_group=pre, post_group=post,
                  conn=bp.connect.All2All(),
                  monitors=['s'], delay=1.5)

You can specify the synapse behavior by using ``syn.runner.set_schedule``.

In [17]:
syn.runner.set_schedule(['input', 'update', 'output', 'monitor'])

Note that you cannot run the synapse connection (unlike neuron groups). You have to run them in a network.

In [18]:
net = bp.Network(pre, syn, post)

net.run(duration=100., inputs=(pre, "ST.input", 20.))